# Zillow SVM Final Submission Notebook

## Script 1 : Data Processing

- Parse unstructured description data
  - normalize capital letters -> lowercase letters
  - normalize commas -> periods
- Extract features : sqft, bedrooms, bathrooms, year
  - sqft (int)
  - bedrooms (int)
  - bathrooms (float)
  - year (int normalized to 1900 (years since 1900))
- Combine with labels : Lead (ppb), Lead level
  - Lead ppb (uint)
  - Lead level (uint 0:3 (treated as categories 0:3))

## Result : clean_data.csv

Features:
- 'desc'
- 'sqft'
- 'bedrooms'
- 'bathrooms'
- 'year'

Labels:
- 'Lead (ppb)'
- 'Lead level'

In [ ]:
import pandas as pd
import re

def normalize_desc(description):
    description = str(description).lower()
    description = description.replace(',', '.')
    return description

def extract_data(description):
    pattern_1 = r"this (\d+) square foot single family home has (\d+(?:\.\d+)?) bedrooms and (\d+(?:\.\d+)?) bathrooms\. this home is located at (.+)"
    pattern_2 = r"(.*?) is a single family home that contains (\d+) sq ft and was built in \d{4}\. it contains (\d+) bedrooms and (\d+) bathrooms"

    match_original = re.search(pattern_1, description)
    if match_original:
        return {
            'sqft': int(match_original.group(1)),
            'bedrooms': int(match_original.group(2)),
            'bathrooms': float(match_original.group(3)),
            'address': match_original.group(4),
            'year': -1  # Year is not included in this format
        }
    
    match_new = re.search(pattern_2, description)
    if match_new:
        return {
            'address': match_new.group(1),
            'sqft': int(match_new.group(2)),
            'bedrooms': int(match_new.group(3)),
            'bathrooms': float(match_new.group(4)),
            'year': -1  # Year is not included in this format
        }

    # Flexible patterns to extract features loosely
    sqft_pattern = r"(\d+)\s*(sq ft|square feet|square foot)"
    bedroom_pattern = r"(\d+)\s*(bedroom|bedrooms|bed)"
    bathroom_pattern = r"(\d+)\s*(bathroom|bathrooms|bath)"
    year_pattern = r"built in (\d{4})"

    # Extract sqft, bedrooms, bathrooms, and year
    sqft = re.search(sqft_pattern, description)
    bedrooms = re.search(bedroom_pattern, description)
    bathrooms = re.search(bathroom_pattern, description)
    year = re.search(year_pattern, description)

    # Format output based on findings or set to -1 if nothing found
    return {
        'sqft': int(sqft.group(1)) if sqft else -1,
        'bedrooms': int(bedrooms.group(1)) if bedrooms else -1,
        'bathrooms': float(bathrooms.group(1)) if bathrooms else -1,
        'year': int(year.group(1)) - 1900 if year else -1, # years normalized to 1900
        'address': None
    }

def process_csv(input_filepath, output_filepath):

    df = pd.read_csv(input_filepath)
    data = []
    
    for index, row in df.iterrows():
        description = normalize_desc(row['Description'])
        extracted = extract_data(description)
        if extracted:
            # append the data
            data.append({
                'desc': description,
                'sqft': extracted['sqft'],
                'bedrooms': extracted['bedrooms'],
                'bathrooms': extracted['bathrooms'],
                'year': extracted['year'],
                'Lead (ppb)': int(row['Lead (ppb)']),
                'Lead level': int(row['Lead level']),
            })
    
    new_df = pd.DataFrame(data)
    new_df.to_csv(output_filepath, index=False)
    print(f'Generated {output_filepath} from {input_filepath}')

input_filepath = 'Zillow_Data.csv'
output_filepath = 'clean_data.csv'
process_csv(input_filepath, output_filepath)

## Feature Visualization

- Scatter plot matrix for sqft, bedrooms, bathrooms
- Bar plot comparing Lead (ppb) to Lead level
- 3D Scatter plot for sqft, bedrooms, bathrooms
- Scatter plot for sqft vs. Lead (ppb) colored by Lead level
- sorted_word_lead_levels.csv : word/average Lead level file
- Bar plot comparing number of words to average Lead level 

In [7]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Load the data
data = pd.read_csv('clean_data.csv')

# Scatter plot matrix for sqft, bedrooms, bathrooms
sns.pairplot(data[['sqft', 'bedrooms', 'bathrooms']], diag_kind="hist")
plt.suptitle('Pairwise scatter plots for sqft, bedrooms, and bathrooms')
plt.savefig('scatter_matrix.png')
plt.close()
print('Generated scatter_matrix.png')

# Bar plot comparing Lead (ppb) to Lead level
sns.barplot(x='Lead (ppb)', y='Lead level', data=data)
plt.title('Bar Plot: Lead (ppb) vs. Lead level')
plt.savefig('lead_comparison_bar.png')
plt.close()
print('Generated lead_comparison_bar.png')

# 3D Scatter plot for sqft, bedrooms, bathrooms
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
sc = ax.scatter(data['sqft'], data['bedrooms'], data['bathrooms'], c=data['bathrooms'], cmap='viridis')
plt.colorbar(sc)
ax.set_xlabel('Sqft')
ax.set_ylabel('Bedrooms')
ax.set_zlabel('Bathrooms')
plt.title('3D Scatter Plot for sqft, bedrooms, bathrooms')
plt.savefig('3d_scatter_sqft_bed_bath.png')
plt.close()
print('Generated 3d_scatter_sqft_bed_bath.png')

# Scatter plot for sqft vs. Lead (ppb) colored by Lead level
plt.figure(figsize=(10, 6))
sc = plt.scatter(data['sqft'], data['Lead (ppb)'], c=data['Lead level'], cmap='coolwarm')
plt.colorbar(sc, label='Lead level')
plt.xlabel('Sqft')
plt.ylabel('Lead (ppb)')
plt.title('Scatter Plot: Sqft vs. Lead (ppb) Colored by Lead level')
plt.savefig('scatter_sqft_lead_colored.png')
plt.close()
print('Generated scatter_sqft_lead_colored.png')

data['Lead level'] = data['Lead level'].astype('category')

# Create the pairplot
sns.pairplot(data[['sqft', 'bedrooms', 'bathrooms', 'Lead level']], 
                         hue='Lead level', 
                         diag_kind='kde', 
                         palette='rocket_r',
                         plot_kws={'alpha': 0.7})

# Additional customization
plt.suptitle('Pairplot of sqft, bedrooms, bathrooms by Lead level', y=1.03)
plt.savefig('new_scatter_matrix.png')
plt.close()
print('Generated new_scatter_matrix.png')

df = pd.read_csv('Zillow_Data.csv')

# Convert 'Description' to string
df['Description'] = df['Description'].astype(str)

# Tokenize the 'Description' column and remove stopwords
stop_words = set(stopwords.words('english'))
df['Tokens'] = df['Description'].apply(lambda x: [word.lower() for word in word_tokenize(x) if word.isalpha() and word.lower() not in stop_words])

# Explode the DataFrame on the 'Tokens' column to associate each word with the corresponding 'Lead level'
exploded_df = df.explode('Tokens')

# Group by each word and calculate the average 'Lead level'
word_lead_levels = exploded_df.groupby('Tokens')['Lead level'].mean().reset_index()

# Sort the DataFrame by 'Lead level' in descending order
word_lead_levels = word_lead_levels.sort_values(by=['Lead level', 'Tokens'], ascending=[False, True])

# Save the sorted result to a CSV file
word_lead_levels.to_csv('sorted_word_lead_levels.csv', index=False)

df = pd.read_csv('sorted_word_lead_levels.csv')

df['Rounded Lead Level'] = np.round(df['Lead level'] * 8) / 8

# Aggregate data to count the number of words for each rounded 'Lead level'
lead_level_counts = df.groupby('Rounded Lead Level').size()

# Create a bar plot
plt.figure(figsize=(10, 6))
lead_level_counts.plot(kind='bar', color='blue')
plt.title('Number of Words at Each Average Lead Level')
plt.xlabel('Average Lead Level')
plt.ylabel('Number of Words')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('word_distribution.png')
plt.close()
print('Generated word_distribution.png')

Generated scatter_matrix.png
Generated lead_comparison_bar.png
Generated 3d_scatter_sqft_bed_bath.png
Generated scatter_sqft_lead_colored.png
Generated new_scatter_matrix.png
Generated word_distribution.png


## SVM Model : Word2Vec Embeddings

- Tokenizes descriptions
- Transforms those tokenizations into 100-Dimensional tensors (word2vec_models)
- Trains SVM to predict Lead level

### Results : 79.4% accuracy

#### Improvements

**Manual Mode** : input descriptions for individual Lead level prediction

#### Shortcomings

**Overfitting Data** : the dataset contains the following distributions of 'Lead levels'

- Lead level 0: 391 entries
- Lead level 1: 55 entries
- Lead level 3: 40 entries
- Lead level 2: 24 entries

Only 30.4% of entries are non-zero labels. 69.6% of entries have 0 Lead level.

This reflects the real-life distribution of Lead levels, but results in a model that overpredicts zero due to a lack of diversified alternative entries.

In [10]:
import pandas as pd
import re
from gensim.models import Word2Vec
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize

# Load data from CSV
df = pd.read_csv('Zillow_Data.csv')

# Tokenization and Word2Vec embedding
def tokenize(description):
    return word_tokenize(description.lower())

tokenized_descriptions = df['Description'].apply(tokenize)
word2vec_model = Word2Vec(sentences=tokenized_descriptions, vector_size=100, window=5, min_count=1, workers=4)

# Aggregate embeddings
def aggregate_embedding(description, model):
    tokens = tokenize(description)
    embeddings = [model.wv[token] for token in tokens if token in model.wv]
    if embeddings:
        return sum(embeddings) / len(embeddings)
    return None

# Aggregate embeddings
df['Embedding'] = df['Description'].apply(lambda x: aggregate_embedding(x, word2vec_model))

# Split data into train and test sets
X = pd.DataFrame(df['Embedding'].tolist())
X.to_csv('description_embedding.csv', index=False)

y = df['Lead level']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM
model = SVC(kernel='linear', C=1.0, random_state=42)
model.fit(X_train, y_train)
print('Trained SVM')

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Process singular input and predict Lead level
description = input('Enter your home listing description here:\n')
embedding = aggregate_embedding(description, word2vec_model)

# Predict the lead level
if embedding is not None:
    X = pd.DataFrame([embedding.tolist()])
    model.fit(X_train, y_train)
    lead_level = model.predict(X)[0]
    print(f'Predicted lead level: {lead_level}')
else:
    print("Unable to predict lead level for this description.")


Trained SVM
Accuracy: 0.7941176470588235
Predicted lead level: 0


## SVM Model : Word2Vec Embeddings + Feature Processing

- Tokenizes descriptions
- Transforms those tokenizations into 100-Dimensional tensors (word2vec_models)
- Combines description tensor with 4 extracted features (sqft, bedrooms, bathrooms, year built)
- Trains SVM to predict Lead level

### Results : 79.4% accuracy

#### Improvements

Additional extracted features included in model (sqft, bedrooms, bathrooms, year built)

- normalized to range (0:4)
  - shift to positive weights (-1 default case set to zero, otherwise positive)
  - weighted 2x relative to word2vec tensor range (-1:1)

#### Shortcomings

**Overfitting Data** : the dataset contains the following distributions of 'Lead levels'

- Lead level 0: 391 entries
- Lead level 1: 55 entries
- Lead level 3: 40 entries
- Lead level 2: 24 entries

Only 30.4% of entries are non-zero labels. 69.6% of entries have 0 Lead level.

This reflects the real-life distribution of Lead levels, but results in a model that overpredicts zero.

**Maladjusted Feature Weights** : SVM trained on description broken into 100-D tensor, plus 4 additional features.

Attempt at normalizing (sqft, bedrooms, bathrooms, year built) using SKLearn MinMaxScaler. Normalized to 2x string embedding vector range to increase weight. Shifted to account for missing extracted features (default -1).

Model has 104 features to process. It cannot process the string description in its entirety while simultaneously weighing the extracted features equivalently. 

**No Increased Accuracy** : The model without the additional features performs similarly, indicating no improvements.

Either:

- Model fails to capture and weigh the additional features properly
- Features do not contain any differentiating information for the model to decide on

In [11]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from nltk.tokenize import word_tokenize

df = pd.read_csv('clean_data.csv')

# nltk tokenization 
def tokenize(description):
    return word_tokenize(description)

# word2vec model from tokenized descriptions
tokenized_descriptions = df['desc'].apply(tokenize)
word2vec_model = Word2Vec(sentences=tokenized_descriptions, vector_size=100, window=5, min_count=1, workers=4)

# Aggregate embeddings
def aggregate_embedding(description, model):
    tokens = tokenize(description)
    embeddings = [model.wv[token] for token in tokens if token in model.wv]
    if embeddings:
        return sum(embeddings) / len(embeddings)
    return None

# Aggregate embeddings
df['Embedding'] = df['desc'].apply(lambda x: aggregate_embedding(x, word2vec_model))

# Select additional features
X_additional = df[['sqft', 'bedrooms', 'bathrooms', 'year']]
X_additional = X_additional.rename(columns={'sqft': 100, 'bedrooms': 101, 'bathrooms': 102, 'year': 103})

# Scale the additional features
scaler = MinMaxScaler(feature_range=(0, 4)) # scaled at 2x relative to word2vec embeddings
X_additional_scaled = scaler.fit_transform(X_additional)
X_additional_scaled = pd.DataFrame(X_additional_scaled, columns=X_additional.columns)

# Combine embeddings with additional features
X = pd.concat([pd.DataFrame(df['Embedding'].tolist()), X_additional_scaled], axis=1)
X.to_csv('combined_features.csv', index=False)

# Split data into train and test sets
y = df['Lead level']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM
model = SVC(kernel='linear', C=1.0, random_state=42)
model.fit(X_train, y_train)
print('Trained SVM')

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Trained SVM
Accuracy: 0.7941176470588235
